In [1]:
!pip install pandas

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 981.9 kB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 41.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 31.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.8/347.8 kB 28.8 MB/s eta 0:00:00

[notice] A new release of pip is available: 23.3.1 -> 25.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
import numpy as np
import pandas as pd
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader, random_split
import torch.nn.functional as F
import torchvision.models as models
import time

In [3]:
# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
print(device)

cuda


In [5]:
# Data augmentation for training
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.RandomCrop(32, padding=4),  # Randomly crop the image
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# No augmentation for validation and test
val_test_transform = transforms.Compose([
    transforms.Resize(224),  # Resize to 224x224 for ResNet
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# Load CIFAR-10 dataset
train_val_dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=train_transform)
test_dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=val_test_transform)

# Split train_val_dataset into train and validation sets (80% train, 20% validation)
train_size = int(0.8 * len(train_val_dataset))
val_size = len(train_val_dataset) - train_size
train_dataset, val_dataset = random_split(train_val_dataset, [train_size, val_size])

# Apply val_test_transform to the validation set
val_dataset.dataset.transform = val_test_transform

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=4)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, num_workers=4)

100.0%


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
# Load pretrained ResNet-50 (Teacher Model)
teacher = models.resnet50(pretrained=True)

# Modify the final fully connected layer for 10 classes (CIFAR-10)
teacher.fc = nn.Linear(teacher.fc.in_features, 10)
# Move models to device
teacher = teacher.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
24.0%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.

In [9]:

model_path = 'Best_Teacher.pth'
# Load the model weights
teacher.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [10]:
# Load pretrained ResNet-18 (Student Model)
student = models.resnet18(pretrained=True)
# Modify the final fully connected layer for 10 classes (CIFAR-10)
student.fc = nn.Linear(student.fc.in_features, 10)
student = student.to(device)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
52.5%IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [11]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [12]:
# CA-KLD Loss for Classification
def cakld_loss(student_logits, teacher_logits, beta_prob):
    # Forward KL (student || teacher)
    student_log_prob = F.log_softmax(student_logits, dim=1)
    teacher_prob = F.softmax(teacher_logits, dim=1)
    forward_kl = F.kl_div(student_log_prob, teacher_prob, reduction='batchmean')

    # Reverse KL (teacher || student)
    teacher_log_prob = F.log_softmax(teacher_logits, dim=1)
    student_prob = F.softmax(student_logits, dim=1)
    reverse_kl = F.kl_div(teacher_log_prob, student_prob, reduction='batchmean')

    # Combined KL loss
    kl_loss = beta_prob * reverse_kl + (1 - beta_prob) * forward_kl
    return kl_loss


In [28]:
def evaluate(model, test_loader, device):
    model = model.to(device)  # Ensure model is on the correct device
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total


In [26]:
def calculate_sparsity(model):
    total_zeros = 0
    total_params = 0
    for name, param in model.named_parameters():
        if 'weight' in name:
            total_zeros += torch.sum(param == 0).item()
            total_params += param.numel()
    return total_zeros / total_params

In [22]:
import torch
import time
def measure_inference_time(model, test_loader, num_runs=5):
    device = torch.device('cpu')
    model.eval()
    model.to(device)

    # Warm-up (one batch to avoid startup cost)
    with torch.no_grad():
        for inputs, _ in test_loader:
            inputs = inputs.to(device)
            _ = model(inputs)
            break

    total_time = 0
    total_images = 0

    with torch.no_grad():
        for _ in range(num_runs):
            for inputs, _ in test_loader:
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                start_time = time.time()
                _ = model(inputs)
                end_time = time.time()

                total_time += (end_time - start_time)
                total_images += batch_size

    avg_time_per_image = total_time / total_images
    return avg_time_per_image


In [16]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters())

def calculate_model_size(model, filename="temp.pth"):
    torch.save(model.state_dict(), filename)
    size = os.path.getsize(filename) / (1024 * 1024)  # Size in MB
    os.remove(filename)
    return size

def compare_model_sizes(teacher, student, pruned_student):
    # Count parameters
    teacher_params = count_parameters(teacher)
    student_params = count_parameters(student)
    pruned_params = count_parameters(pruned_student)
    
    # Calculate disk size
    teacher_size = calculate_model_size(teacher, "teacher.pth")
    student_size = calculate_model_size(student, "student.pth")
    pruned_size = calculate_model_size(pruned_student, "pruned_student.pth")
    
    # Print comparison
    print("\n--- Model Size Comparison ---")
    print(f"Teacher Model: {teacher_params} parameters, {teacher_size:.2f} MB")
    print(f"Student Model (Before Pruning): {student_params} parameters, {student_size:.2f} MB")
    print(f"Student Model (After Pruning): {pruned_params} parameters, {pruned_size:.2f} MB")
    
    # Calculate compression ratio
    compression_ratio = student_size / pruned_size
    print(f"\nCompression Ratio: {compression_ratio:.2f}x")

In [17]:
def train_model(model, train_loader, val_loader, epochs=10, lr=0.001, patience=3):
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    
    best_val_accuracy = 0.0
    best_model_state = None
    patience_counter = 0  # Counter for early stopping
    
    for epoch in range(epochs):
        print(epoch)
        model.train()
        running_loss = 0.0
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            running_loss += loss.item()
        
        # Evaluate on the validation set
        val_accuracy = evaluate(model, val_loader, device)
        print(f"Epoch {epoch+1}/{epochs} | Loss: {running_loss/len(train_loader):.4f} | Val Accuracy: {val_accuracy:.2f}%")
        
        # Early stopping logic
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            best_model_state = model.state_dict()
            patience_counter = 0  # Reset patience counter
            torch.save(model.state_dict(), 'best_teacher_model.pth')  # Save the best model
            print(f" New best model saved with validation accuracy: {best_val_accuracy:.2f}%")
        else:
            patience_counter += 1
            print(f" No improvement in validation accuracy ({patience_counter}/{patience})")
            
            # Stop training if no improvement for 'patience' epochs
            if patience_counter >= patience:
                print(f"\nEarly stopping triggered! No improvement for {patience} epochs.")
                break
    
    # Load the best model state
    model.load_state_dict(torch.load('best_teacher_model.pth'))
    print("\nLoading the best model for final evaluation.")
    
    # Evaluate on the test set
    test_accuracy = evaluate(model, test_loader, device)
    print(f"Test Accuracy with Best Model: {test_accuracy:.2f}%")
    
    return model



In [18]:
def compute_gradient_importance(
    teacher, student, data_loader, device, temperature=4.0, alpha=0.5, beta_prob=0.5, accumulation_epochs=3
):
    importance_scores = {}

    # Initialize importance score storage for conv layer weights only
    for name, param in student.named_parameters():
        if 'weight' in name and isinstance(param, nn.Parameter) and len(param.shape) == 4:
            importance_scores[name] = torch.zeros_like(param.data, device=device)

    teacher.to(device).eval()
    student.to(device).train()

    for epoch in range(accumulation_epochs):
        print(f"Accumulation Epoch {epoch+1}/{accumulation_epochs}")
        for inputs, labels in data_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            student.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature scaling
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature

            # Compute CA-KLD loss with temperature
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ce_loss = F.cross_entropy(student_logits, labels)

            # Total loss
            loss = alpha * distillation_loss + (1 - alpha) * ce_loss
            loss.backward()

            # Accumulate squared gradients (L2 norm)
            for name, param in student.named_parameters():
                if name in importance_scores and param.grad is not None:
                    importance_scores[name] += (param.grad.detach() ** 2)

    # Normalize importance scores by total batches processed
    total_batches = accumulation_epochs * len(data_loader)
    for name in importance_scores:
        importance_scores[name] /= total_batches

    return importance_scores


In [19]:
def gradient_based_global_prune(model, importance_scores, prune_ratio=0.95):
    all_scores = torch.cat([score.flatten() for score in importance_scores.values()])
    threshold = torch.topk(all_scores, k=int(prune_ratio * all_scores.numel()), largest=False)[0][-1]

    for name, param in model.named_parameters():
        if name in importance_scores:
            mask = (importance_scores[name] > threshold).float()
            param.data.mul_(mask)

    return model


In [20]:
import torch
import torch.nn.functional as F
import torch.optim as optim

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

def retrain_with_sparsity(student, train_loader, val_loader, epochs=5, save_path="retrained_student_model.pt", patience=3):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks AND zero momentum buffers for pruned weights
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:  # Consider only conv layers
            mask = (param != 0).float().to(device)
            masks[name] = mask
            # Zero momentum buffers for pruned weights
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    student = student.to(device)
    best_val_acc = 0.0
    best_model = None
    patience_counter = 0  # Counter for early stopping

    # 2. Add gradient clipping to prevent NaN
    max_grad_norm = 1.0

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = student(inputs)
            loss = F.cross_entropy(outputs, labels)
            loss.backward()

            # Apply masks to gradients
            for name, param in student.named_parameters():
                if name in masks:
                    param.grad.data *= masks[name]

            # Gradient clipping before optimizer step
            torch.nn.utils.clip_grad_norm_(student.parameters(), max_grad_norm)

            optimizer.step()

            # Reapply masks and update momentum buffers
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = outputs.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation phase
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)

                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total

        # Track best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model = student.state_dict()
            torch.save(best_model, save_path)
            patience_counter = 0  # Reset patience counter
            print(f"New best model saved with Val Accuracy: {best_val_acc:.2f}%")
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break  # Stop training

        # Print results
        sparsity = calculate_sparsity(student)
        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}%")
        print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}% | Sparsity: {sparsity*100:.2f}%\n")

    print(f"Best Validation Accuracy: {best_val_acc:.2f}% | Best Model Saved at: {save_path}")
    return student

In [21]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import time

# KD training with CA-KLD loss and mask-based momentum handling
def retrain_with_KD(teacher, student, train_loader, val_loader, epochs=50,
                    temperature=5.0, alpha=0.5, beta_prob=0.5, patience=5,
                    save_path="student_before_pruning.pth"):

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    # 1. Store masks and zero momentum buffers
    masks = {}
    for name, param in student.named_parameters():
        if 'weight' in name and param.dim() == 4:
            mask = (param != 0).float().to(device)
            masks[name] = mask
            if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                optimizer.state[param]['momentum_buffer'] *= mask

    teacher = teacher.to(device).eval()
    student = student.to(device)

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss, correct, total = 0.0, 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # Apply temperature
            teacher_logits_temp = teacher_logits / temperature
            student_logits_temp = student_logits / temperature

            # CA-KLD loss
            kd_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)
            ce_loss = F.cross_entropy(student_logits, labels)

            loss = alpha * kd_loss + (1 - alpha) * ce_loss
            loss.backward()
            optimizer.step()

            # Reapply masks and update momentum
            for name, param in student.named_parameters():
                if name in masks:
                    param.data *= masks[name]
                    if optimizer.state.get(param, None) and 'momentum_buffer' in optimizer.state[param]:
                        optimizer.state[param]['momentum_buffer'] *= masks[name]

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation
        student.eval()
        val_loss, val_correct, val_total = 0.0, 0, 0
        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = student(inputs)
                loss = F.cross_entropy(outputs, labels)
                val_loss += loss.item()
                _, predicted = outputs.max(1)
                val_correct += predicted.eq(labels).sum().item()
                val_total += labels.size(0)

        val_loss /= len(val_loader)
        val_acc = 100.0 * val_correct / val_total
        sparsity = calculate_sparsity(student) * 100.0  # Assuming this function is defined elsewhere

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.2f}% | "
              f"Val Loss: {val_loss:.4f} | Val Acc: {val_acc:.2f}% | Sparsity: {sparsity:.2f}%")

        # Early stopping logic
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Restore and save best model
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")
    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [83]:
import time
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader

# Training function with KD + CA-KLD
def train_kd_pruning(teacher, student, train_loader, val_loader, epochs=50,
                     pruning_type='unstructured', temperature=5.0, alpha=0.5,
                     beta_prob=0.5, patience=5, save_path="student_before_pruning.pth"):
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    optimizer = optim.SGD(student.parameters(), lr=0.01, momentum=0.9)

    teacher = teacher.to(device)
    student = student.to(device)
    teacher.eval()  # Freeze teacher

    best_val_acc = 0.0
    best_model_state = None
    patience_counter = 0
    start_time = time.time()

    for epoch in range(epochs):
        student.train()
        total_loss = 0.0
        correct, total = 0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)

            with torch.no_grad():
                teacher_logits = teacher(inputs)

            student_logits = student(inputs)

            # CA-KLD loss (temperature-scaled)
            student_logits_temp = student_logits / temperature
            teacher_logits_temp = teacher_logits / temperature
            distillation_loss = cakld_loss(student_logits_temp, teacher_logits_temp, beta_prob) * (temperature ** 2)

            # Cross-entropy loss
            ground_truth_loss = F.cross_entropy(student_logits, labels)

            # Combined loss
            loss = alpha * distillation_loss + (1 - alpha) * ground_truth_loss

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            _, predicted = student_logits.max(1)
            correct += predicted.eq(labels).sum().item()
            total += labels.size(0)

        train_loss = total_loss / len(train_loader)
        train_acc = 100.0 * correct / total

        # Validation accuracy
        val_acc = evaluate(student, val_loader, device)

        print(f"Epoch {epoch+1}/{epochs} | Train Loss: {train_loss:.4f} | "
              f"Train Acc: {train_acc:.2f}% | Val Acc: {val_acc:.2f}%")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_state = student.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping triggered at epoch {epoch+1}. No improvement for {patience} epochs.")
                break

    # Load best model state and save
    student.load_state_dict(best_model_state)
    torch.save(student.state_dict(), save_path)
    print(f"Student model saved before pruning at: {save_path}")

    total_time = time.time() - start_time
    print(f"Total Training Time: {total_time // 60:.0f}m {total_time % 60:.0f}s")

    return student

In [29]:
# Calculate sparsity
sparsity = calculate_sparsity(student)
print(f"Sparsity Before Pruning: {sparsity * 100:.2f}%")

teacher_accuracy = evaluate(teacher, test_loader, device)
student_accuracy = evaluate(student, test_loader, device)
print(f"Teacher Model Test Accuracy: {teacher_accuracy:.2f}%")
print(f"Student Model Test Accuracy Before Pruning: {student_accuracy:.2f}%")

Sparsity Before Pruning: 0.00%
Teacher Model Test Accuracy: 95.40%
Student Model Test Accuracy Before Pruning: 95.29%


## 93% Sparsity

In [30]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [31]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [32]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%


In [33]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 35.66%
Epoch 1/200 | Train Loss: 2.0993 | Train Acc: 19.77%
Validation Loss: 1.6356 | Validation Acc: 35.66% | Sparsity: 93.92%

New best model saved with Val Accuracy: 55.56%
Epoch 2/200 | Train Loss: 1.3603 | Train Acc: 48.59%
Validation Loss: 1.1900 | Validation Acc: 55.56% | Sparsity: 93.92%

New best model saved with Val Accuracy: 73.77%
Epoch 3/200 | Train Loss: 0.8713 | Train Acc: 68.97%
Validation Loss: 0.7565 | Validation Acc: 73.77% | Sparsity: 93.92%

New best model saved with Val Accuracy: 78.06%
Epoch 4/200 | Train Loss: 0.6204 | Train Acc: 78.50%
Validation Loss: 0.6466 | Validation Acc: 78.06% | Sparsity: 93.92%

New best model saved with Val Accuracy: 81.19%
Epoch 5/200 | Train Loss: 0.4840 | Train Acc: 83.36%
Validation Loss: 0.5528 | Validation Acc: 81.19% | Sparsity: 93.92%

New best model saved with Val Accuracy: 83.83%
Epoch 6/200 | Train Loss: 0.3920 | Train Acc: 86.78%
Validation Loss: 0.4883 | Validation Acc: 83.83% | Spar

In [34]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 84.44%


In [36]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [37]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [38]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 6.3164 | Train Acc: 41.28% | Val Loss: 1.5014 | Val Acc: 58.30% | Sparsity: 93.92%
Epoch 2/50 | Train Loss: 3.0966 | Train Acc: 74.76% | Val Loss: 0.9108 | Val Acc: 76.63% | Sparsity: 93.92%
Epoch 3/50 | Train Loss: 2.0570 | Train Acc: 83.55% | Val Loss: 0.6739 | Val Acc: 81.50% | Sparsity: 93.92%
Epoch 4/50 | Train Loss: 1.6038 | Train Acc: 87.52% | Val Loss: 0.6448 | Val Acc: 83.85% | Sparsity: 93.92%
Epoch 5/50 | Train Loss: 1.3316 | Train Acc: 89.84% | Val Loss: 0.5693 | Val Acc: 85.33% | Sparsity: 93.92%
Epoch 6/50 | Train Loss: 1.1241 | Train Acc: 91.87% | Val Loss: 0.4796 | Val Acc: 86.54% | Sparsity: 93.92%
Epoch 7/50 | Train Loss: 0.9732 | Train Acc: 93.22% | Val Loss: 0.4670 | Val Acc: 87.70% | Sparsity: 93.92%
Epoch 8/50 | Train Loss: 0.8462 | Train Acc: 94.60% | Val Loss: 0.5204 | Val Acc: 86.63% | Sparsity: 93.92%
Epoch 9/50 | Train Loss: 0.7445 | Train Acc: 95.50% | Val Loss: 0.4442 | Val Acc: 88.27% | Sparsity: 93.92%
Epoch 10/50 | Train Loss: 0.

In [39]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 90.55%


In [40]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [41]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=7.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.94)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [42]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=7.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 6.5293 | Train Acc: 32.02% | Val Loss: 1.6913 | Val Acc: 54.95% | Sparsity: 93.92%
Epoch 2/50 | Train Loss: 2.6744 | Train Acc: 76.64% | Val Loss: 1.1272 | Val Acc: 73.60% | Sparsity: 93.92%
Epoch 3/50 | Train Loss: 1.6963 | Train Acc: 85.89% | Val Loss: 0.5245 | Val Acc: 85.56% | Sparsity: 93.92%
Epoch 4/50 | Train Loss: 1.3119 | Train Acc: 89.51% | Val Loss: 0.4228 | Val Acc: 87.49% | Sparsity: 93.92%
Epoch 5/50 | Train Loss: 1.0943 | Train Acc: 91.70% | Val Loss: 0.5385 | Val Acc: 85.99% | Sparsity: 93.92%
Epoch 6/50 | Train Loss: 0.9378 | Train Acc: 93.20% | Val Loss: 0.5076 | Val Acc: 87.40% | Sparsity: 93.92%
Epoch 7/50 | Train Loss: 0.8127 | Train Acc: 94.62% | Val Loss: 0.3619 | Val Acc: 89.87% | Sparsity: 93.92%
Epoch 8/50 | Train Loss: 0.7068 | Train Acc: 95.63% | Val Loss: 0.3803 | Val Acc: 90.02% | Sparsity: 93.92%
Epoch 9/50 | Train Loss: 0.6250 | Train Acc: 96.69% | Val Loss: 0.3600 | Val Acc: 90.05% | Sparsity: 93.92%
Epoch 10/50 | Train Loss: 0.

In [43]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 91.37%


# 90% of Sparsity

In [44]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [45]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7,beta_prob=0.5, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()
pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)
total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [46]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 64.24%
Epoch 1/200 | Train Loss: 1.6893 | Train Acc: 36.20%
Validation Loss: 1.0352 | Validation Acc: 64.24% | Sparsity: 90.00%

New best model saved with Val Accuracy: 85.56%
Epoch 2/200 | Train Loss: 0.6579 | Train Acc: 77.54%
Validation Loss: 0.4411 | Validation Acc: 85.56% | Sparsity: 90.00%

New best model saved with Val Accuracy: 89.20%
Epoch 3/200 | Train Loss: 0.3260 | Train Acc: 89.28%
Validation Loss: 0.3294 | Validation Acc: 89.20% | Sparsity: 90.00%

New best model saved with Val Accuracy: 90.34%
Epoch 4/200 | Train Loss: 0.2119 | Train Acc: 92.95%
Validation Loss: 0.2968 | Validation Acc: 90.34% | Sparsity: 90.00%

New best model saved with Val Accuracy: 90.81%
Epoch 5/200 | Train Loss: 0.1453 | Train Acc: 95.12%
Validation Loss: 0.2869 | Validation Acc: 90.81% | Sparsity: 90.00%

New best model saved with Val Accuracy: 91.20%
Epoch 6/200 | Train Loss: 0.0993 | Train Acc: 96.77%
Validation Loss: 0.2829 | Validation Acc: 91.20% | Spar

In [47]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(After Retrain): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(After Retrain): 91.22%


In [87]:

pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
print(f"Pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")


Pruned Student Model Inference Time: 1.65 ms per batch


# Retrained with KD

In [48]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [49]:
student_accuracy = evaluate(student, test_loader, device)
print(f"Student Model Test Accuracy: {student_accuracy:.2f}%")

Student Model Test Accuracy: 95.29%


In [50]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [51]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 4.7247 | Train Acc: 58.00% | Val Loss: 0.6383 | Val Acc: 82.47% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 1.5648 | Train Acc: 88.30% | Val Loss: 0.3718 | Val Acc: 89.63% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 1.0694 | Train Acc: 92.52% | Val Loss: 0.4017 | Val Acc: 89.29% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 0.8279 | Train Acc: 94.85% | Val Loss: 0.3302 | Val Acc: 91.30% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 0.6732 | Train Acc: 96.28% | Val Loss: 0.2834 | Val Acc: 91.88% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 0.5648 | Train Acc: 97.31% | Val Loss: 0.2586 | Val Acc: 92.78% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 0.4823 | Train Acc: 98.03% | Val Loss: 0.2369 | Val Acc: 93.30% | Sparsity: 90.00%
Epoch 8/50 | Train Loss: 0.4202 | Train Acc: 98.40% | Val Loss: 0.2305 | Val Acc: 93.51% | Sparsity: 90.00%
Epoch 9/50 | Train Loss: 0.3819 | Train Acc: 98.75% | Val Loss: 0.2173 | Val Acc: 93.52% | Sparsity: 90.00%
Epoch 10/50 | Train Loss: 0.

In [52]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 92.65%


In [53]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [54]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=7.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.9008)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [55]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=7.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 4.0696 | Train Acc: 61.09% | Val Loss: 0.4792 | Val Acc: 85.92% | Sparsity: 90.00%
Epoch 2/50 | Train Loss: 1.3172 | Train Acc: 89.75% | Val Loss: 0.4438 | Val Acc: 87.99% | Sparsity: 90.00%
Epoch 3/50 | Train Loss: 0.9349 | Train Acc: 93.31% | Val Loss: 0.3550 | Val Acc: 89.55% | Sparsity: 90.00%
Epoch 4/50 | Train Loss: 0.7432 | Train Acc: 95.40% | Val Loss: 0.2599 | Val Acc: 92.43% | Sparsity: 90.00%
Epoch 5/50 | Train Loss: 0.5919 | Train Acc: 96.84% | Val Loss: 0.2599 | Val Acc: 92.61% | Sparsity: 90.00%
Epoch 6/50 | Train Loss: 0.5124 | Train Acc: 97.75% | Val Loss: 0.2281 | Val Acc: 93.56% | Sparsity: 90.00%
Epoch 7/50 | Train Loss: 0.4535 | Train Acc: 98.16% | Val Loss: 0.2356 | Val Acc: 93.19% | Sparsity: 90.00%
Epoch 8/50 | Train Loss: 0.4020 | Train Acc: 98.60% | Val Loss: 0.2371 | Val Acc: 92.98% | Sparsity: 90.00%
Epoch 9/50 | Train Loss: 0.3615 | Train Acc: 98.87% | Val Loss: 0.2121 | Val Acc: 93.79% | Sparsity: 90.00%
Epoch 10/50 | Train Loss: 0.

In [56]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 93.05%


# 80% Sparsity

In [61]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [62]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [63]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%


In [64]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_80.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 93.86%
Epoch 1/100 | Train Loss: 0.4171 | Train Acc: 86.99%
Validation Loss: 0.1858 | Validation Acc: 93.86% | Sparsity: 80.00%

New best model saved with Val Accuracy: 94.57%
Epoch 2/100 | Train Loss: 0.1117 | Train Acc: 96.42%
Validation Loss: 0.1717 | Validation Acc: 94.57% | Sparsity: 80.00%

New best model saved with Val Accuracy: 95.18%
Epoch 3/100 | Train Loss: 0.0552 | Train Acc: 98.25%
Validation Loss: 0.1580 | Validation Acc: 95.18% | Sparsity: 80.00%

New best model saved with Val Accuracy: 95.54%
Epoch 4/100 | Train Loss: 0.0304 | Train Acc: 99.11%
Validation Loss: 0.1562 | Validation Acc: 95.54% | Sparsity: 80.00%

Epoch 5/100 | Train Loss: 0.0181 | Train Acc: 99.49%
Validation Loss: 0.1795 | Validation Acc: 95.16% | Sparsity: 80.00%

Epoch 6/100 | Train Loss: 0.0134 | Train Acc: 99.59%
Validation Loss: 0.1889 | Validation Acc: 94.91% | Sparsity: 80.00%

Epoch 7/100 | Train Loss: 0.0105 | Train Acc: 99.72%
Validation Loss: 0.1887 | V

In [65]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain without KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain without KD): 93.90%


In [ ]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
print(f"Student Model Inference Time(After Pruning): {pruned_student_inference_time * 1000:.2f} ms per batch")

In [66]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [67]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [68]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.6337 | Train Acc: 88.75% | Val Loss: 0.2158 | Val Acc: 93.18% | Sparsity: 80.00%
Epoch 2/50 | Train Loss: 0.7188 | Train Acc: 95.72% | Val Loss: 0.2001 | Val Acc: 93.75% | Sparsity: 80.00%
Epoch 3/50 | Train Loss: 0.5160 | Train Acc: 97.69% | Val Loss: 0.2020 | Val Acc: 93.92% | Sparsity: 80.00%
Epoch 4/50 | Train Loss: 0.4023 | Train Acc: 98.49% | Val Loss: 0.1446 | Val Acc: 95.57% | Sparsity: 80.00%
Epoch 5/50 | Train Loss: 0.3328 | Train Acc: 98.89% | Val Loss: 0.1401 | Val Acc: 95.53% | Sparsity: 80.00%
Epoch 6/50 | Train Loss: 0.2912 | Train Acc: 99.08% | Val Loss: 0.1239 | Val Acc: 96.09% | Sparsity: 80.00%
Epoch 7/50 | Train Loss: 0.2643 | Train Acc: 99.16% | Val Loss: 0.1180 | Val Acc: 96.33% | Sparsity: 80.00%
Epoch 8/50 | Train Loss: 0.2411 | Train Acc: 99.17% | Val Loss: 0.1250 | Val Acc: 95.94% | Sparsity: 80.00%
Epoch 9/50 | Train Loss: 0.2202 | Train Acc: 99.27% | Val Loss: 0.1248 | Val Acc: 95.98% | Sparsity: 80.00%
Epoch 10/50 | Train Loss: 0.

In [69]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 94.32%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=7.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.8007)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=7.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.4604 | Train Acc: 89.33% | Val Loss: 0.2326 | Val Acc: 92.67% | Sparsity: 80.00%
Epoch 2/50 | Train Loss: 0.6872 | Train Acc: 95.98% | Val Loss: 0.1875 | Val Acc: 94.08% | Sparsity: 80.00%
Epoch 3/50 | Train Loss: 0.5078 | Train Acc: 97.77% | Val Loss: 0.1284 | Val Acc: 95.96% | Sparsity: 80.00%
Epoch 4/50 | Train Loss: 0.4008 | Train Acc: 98.52% | Val Loss: 0.1405 | Val Acc: 95.60% | Sparsity: 80.00%
Epoch 5/50 | Train Loss: 0.3291 | Train Acc: 99.02% | Val Loss: 0.1340 | Val Acc: 95.76% | Sparsity: 80.00%
Epoch 6/50 | Train Loss: 0.2938 | Train Acc: 99.08% | Val Loss: 0.1273 | Val Acc: 96.04% | Sparsity: 80.00%
Epoch 7/50 | Train Loss: 0.2672 | Train Acc: 99.22% | Val Loss: 0.1228 | Val Acc: 96.14% | Sparsity: 80.00%
Epoch 8/50 | Train Loss: 0.2415 | Train Acc: 99.28% | Val Loss: 0.1283 | Val Acc: 96.05% | Sparsity: 80.00%
Epoch 9/50 | Train Loss: 0.2327 | Train Acc: 99.26% | Val Loss: 0.1121 | Val Acc: 96.57% | Sparsity: 80.00%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Pruned Student Model Test Accuracy(Retrain with KD): {student_accuracy:.2f}%")

Pruned Student Model Test Accuracy(Retrain with KD): 94.24%


# 75% of Sparsity

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=7.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_75.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 95.98%
Epoch 1/100 | Train Loss: 0.2448 | Train Acc: 92.87%
Validation Loss: 0.1245 | Validation Acc: 95.98% | Sparsity: 75.00%

Epoch 2/100 | Train Loss: 0.0720 | Train Acc: 97.82%
Validation Loss: 0.1296 | Validation Acc: 95.81% | Sparsity: 75.00%

New best model saved with Val Accuracy: 96.04%
Epoch 3/100 | Train Loss: 0.0297 | Train Acc: 99.14%
Validation Loss: 0.1348 | Validation Acc: 96.04% | Sparsity: 75.00%

New best model saved with Val Accuracy: 96.16%
Epoch 4/100 | Train Loss: 0.0174 | Train Acc: 99.48%
Validation Loss: 0.1330 | Validation Acc: 96.16% | Sparsity: 75.00%

New best model saved with Val Accuracy: 96.26%
Epoch 5/100 | Train Loss: 0.0109 | Train Acc: 99.74%
Validation Loss: 0.1487 | Validation Acc: 96.26% | Sparsity: 75.00%

New best model saved with Val Accuracy: 96.62%
Epoch 6/100 | Train Loss: 0.0060 | Train Acc: 99.84%
Validation Loss: 0.1332 | Validation Acc: 96.62% | Sparsity: 75.00%

Epoch 7/100 | Train Loss: 0.0043 

In [ ]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 94.34%


In [121]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(retrained_student, test_loader,)
print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 1.70 ms per batch


# Retrain with KD

In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")
student = student.to(device)


Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.2571 | Train Acc: 92.11% | Val Loss: 0.1841 | Val Acc: 94.36% | Sparsity: 75.00%
Epoch 2/50 | Train Loss: 0.6186 | Train Acc: 96.75% | Val Loss: 0.1615 | Val Acc: 95.17% | Sparsity: 75.00%
Epoch 3/50 | Train Loss: 0.4222 | Train Acc: 98.38% | Val Loss: 0.1287 | Val Acc: 96.00% | Sparsity: 75.00%
Epoch 4/50 | Train Loss: 0.3290 | Train Acc: 98.94% | Val Loss: 0.1075 | Val Acc: 96.67% | Sparsity: 75.00%
Epoch 5/50 | Train Loss: 0.2901 | Train Acc: 99.15% | Val Loss: 0.1116 | Val Acc: 96.43% | Sparsity: 75.00%
Epoch 6/50 | Train Loss: 0.2570 | Train Acc: 99.19% | Val Loss: 0.1011 | Val Acc: 96.80% | Sparsity: 75.00%
Epoch 7/50 | Train Loss: 0.2257 | Train Acc: 99.25% | Val Loss: 0.1003 | Val Acc: 96.63% | Sparsity: 75.00%
Epoch 8/50 | Train Loss: 0.2045 | Train Acc: 99.31% | Val Loss: 0.0960 | Val Acc: 96.78% | Sparsity: 75.00%
Epoch 9/50 | Train Loss: 0.1923 | Train Acc: 99.27% | Val Loss: 0.0947 | Val Acc: 96.90% | Sparsity: 75.00%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 94.88%


In [126]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 2.00 ms per batch


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.75062)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.2503 | Train Acc: 92.19% | Val Loss: 0.1845 | Val Acc: 94.03% | Sparsity: 75.00%
Epoch 2/50 | Train Loss: 0.6055 | Train Acc: 96.87% | Val Loss: 0.1313 | Val Acc: 95.95% | Sparsity: 75.00%
Epoch 3/50 | Train Loss: 0.4348 | Train Acc: 98.31% | Val Loss: 0.1207 | Val Acc: 96.15% | Sparsity: 75.00%
Epoch 4/50 | Train Loss: 0.3382 | Train Acc: 98.85% | Val Loss: 0.1063 | Val Acc: 96.59% | Sparsity: 75.00%
Epoch 5/50 | Train Loss: 0.2738 | Train Acc: 99.18% | Val Loss: 0.1029 | Val Acc: 96.71% | Sparsity: 75.00%
Epoch 6/50 | Train Loss: 0.2436 | Train Acc: 99.23% | Val Loss: 0.1030 | Val Acc: 96.72% | Sparsity: 75.00%
Epoch 7/50 | Train Loss: 0.2246 | Train Acc: 99.23% | Val Loss: 0.0994 | Val Acc: 96.76% | Sparsity: 75.00%
Epoch 8/50 | Train Loss: 0.2069 | Train Acc: 99.28% | Val Loss: 0.0974 | Val Acc: 96.78% | Sparsity: 75.00%
Epoch 9/50 | Train Loss: 0.1932 | Train Acc: 99.31% | Val Loss: 0.0950 | Val Acc: 96.84% | Sparsity: 75.00%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 94.87%


# 70% Sparsity

In [125]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [126]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.707)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [127]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=100,  save_path='retrained_student_model_70.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 97.11%
Epoch 1/100 | Train Loss: 0.1764 | Train Acc: 95.05%
Validation Loss: 0.0948 | Validation Acc: 97.11% | Sparsity: 70.64%

Epoch 2/100 | Train Loss: 0.0499 | Train Acc: 98.53%
Validation Loss: 0.0942 | Validation Acc: 96.98% | Sparsity: 70.64%

Epoch 3/100 | Train Loss: 0.0213 | Train Acc: 99.38%
Validation Loss: 0.0966 | Validation Acc: 97.10% | Sparsity: 70.64%

Epoch 4/100 | Train Loss: 0.0098 | Train Acc: 99.77%
Validation Loss: 0.1005 | Validation Acc: 97.07% | Sparsity: 70.64%

New best model saved with Val Accuracy: 97.30%
Epoch 5/100 | Train Loss: 0.0059 | Train Acc: 99.86%
Validation Loss: 0.0997 | Validation Acc: 97.30% | Sparsity: 70.64%

Epoch 6/100 | Train Loss: 0.0040 | Train Acc: 99.93%
Validation Loss: 0.1014 | Validation Acc: 97.24% | Sparsity: 70.64%

New best model saved with Val Accuracy: 97.52%
Epoch 7/100 | Train Loss: 0.0022 | Train Acc: 99.95%
Validation Loss: 0.0983 | Validation Acc: 97.52% | Sparsity: 70.64%

Epoch

In [128]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 94.50%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.707)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.0910 | Train Acc: 93.56% | Val Loss: 0.1450 | Val Acc: 95.20% | Sparsity: 70.64%
Epoch 2/50 | Train Loss: 0.5123 | Train Acc: 97.60% | Val Loss: 0.1085 | Val Acc: 96.44% | Sparsity: 70.64%
Epoch 3/50 | Train Loss: 0.3669 | Train Acc: 98.78% | Val Loss: 0.0849 | Val Acc: 97.13% | Sparsity: 70.64%
Epoch 4/50 | Train Loss: 0.3024 | Train Acc: 99.02% | Val Loss: 0.0796 | Val Acc: 97.33% | Sparsity: 70.64%
Epoch 5/50 | Train Loss: 0.2597 | Train Acc: 99.14% | Val Loss: 0.0833 | Val Acc: 97.18% | Sparsity: 70.64%
Epoch 6/50 | Train Loss: 0.2337 | Train Acc: 99.21% | Val Loss: 0.0789 | Val Acc: 97.41% | Sparsity: 70.64%
Epoch 7/50 | Train Loss: 0.2085 | Train Acc: 99.25% | Val Loss: 0.0813 | Val Acc: 97.32% | Sparsity: 70.64%
Epoch 8/50 | Train Loss: 0.1980 | Train Acc: 99.32% | Val Loss: 0.0790 | Val Acc: 97.42% | Sparsity: 70.64%
Epoch 9/50 | Train Loss: 0.1770 | Train Acc: 99.35% | Val Loss: 0.0784 | Val Acc: 97.42% | Sparsity: 70.64%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 94.97%


In [211]:
# Measure inference times
pruned_student_inference_time = measure_inference_time(pruned_student, test_loader, device)
print(f" Retrained pruned Student Model Inference Time: {pruned_student_inference_time * 1000:.2f} ms per batch")

 Retrained pruned Student Model Inference Time: 1.76 ms per batch


In [ ]:
model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=7.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.707)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 34s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=7.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 1.0044 | Train Acc: 94.14% | Val Loss: 0.1237 | Val Acc: 95.91% | Sparsity: 70.64%
Epoch 2/50 | Train Loss: 0.4977 | Train Acc: 97.69% | Val Loss: 0.1097 | Val Acc: 96.42% | Sparsity: 70.64%
Epoch 3/50 | Train Loss: 0.3688 | Train Acc: 98.74% | Val Loss: 0.1008 | Val Acc: 96.78% | Sparsity: 70.64%
Epoch 4/50 | Train Loss: 0.3090 | Train Acc: 99.07% | Val Loss: 0.1140 | Val Acc: 96.32% | Sparsity: 70.64%
Epoch 5/50 | Train Loss: 0.2698 | Train Acc: 99.28% | Val Loss: 0.1038 | Val Acc: 96.51% | Sparsity: 70.64%
Epoch 6/50 | Train Loss: 0.2382 | Train Acc: 99.33% | Val Loss: 0.0738 | Val Acc: 97.60% | Sparsity: 70.64%
Epoch 7/50 | Train Loss: 0.2095 | Train Acc: 99.30% | Val Loss: 0.0761 | Val Acc: 97.49% | Sparsity: 70.64%
Epoch 8/50 | Train Loss: 0.2025 | Train Acc: 99.38% | Val Loss: 0.0715 | Val Acc: 97.57% | Sparsity: 70.64%
Epoch 9/50 | Train Loss: 0.1876 | Train Acc: 99.31% | Val Loss: 0.0874 | Val Acc: 96.95% | Sparsity: 70.64%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 94.88%


In [ ]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [ ]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.707)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 10.00%


In [ ]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model_50%.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 96.62%
Epoch 1/200 | Train Loss: 0.1792 | Train Acc: 94.97%
Validation Loss: 0.1015 | Validation Acc: 96.62% | Sparsity: 70.64%

New best model saved with Val Accuracy: 96.72%
Epoch 2/200 | Train Loss: 0.0501 | Train Acc: 98.45%
Validation Loss: 0.0983 | Validation Acc: 96.72% | Sparsity: 70.64%

Epoch 3/200 | Train Loss: 0.0211 | Train Acc: 99.43%
Validation Loss: 0.1106 | Validation Acc: 96.65% | Sparsity: 70.64%

New best model saved with Val Accuracy: 97.11%
Epoch 4/200 | Train Loss: 0.0102 | Train Acc: 99.75%
Validation Loss: 0.1053 | Validation Acc: 97.11% | Sparsity: 70.64%

Epoch 5/200 | Train Loss: 0.0064 | Train Acc: 99.84%
Validation Loss: 0.1090 | Validation Acc: 96.89% | Sparsity: 70.64%

New best model saved with Val Accuracy: 97.25%
Epoch 6/200 | Train Loss: 0.0044 | Train Acc: 99.91%
Validation Loss: 0.0977 | Validation Acc: 97.25% | Sparsity: 70.64%

Epoch 7/200 | Train Loss: 0.0030 | Train Acc: 99.94%
Validation Loss: 0.0964 | V

In [ ]:
student_accuracy = evaluate(retrained_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 94.60%


## 50% Sparsity

In [113]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [114]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 32s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [115]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f"Student Model Test Accuracy After Pruning: {student_accuracy:.2f}%")

Student Model Test Accuracy After Pruning: 10.07%


In [116]:
start_time = time.time()
retrained_student = retrain_with_sparsity(
    pruned_student, train_loader, val_loader,
    epochs=200,  save_path='retrained_student_model_50%.pt',patience=5
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

New best model saved with Val Accuracy: 98.50%
Epoch 1/200 | Train Loss: 0.0709 | Train Acc: 98.11%
Validation Loss: 0.0517 | Validation Acc: 98.50% | Sparsity: 50.46%

New best model saved with Val Accuracy: 98.68%
Epoch 2/200 | Train Loss: 0.0182 | Train Acc: 99.54%
Validation Loss: 0.0486 | Validation Acc: 98.68% | Sparsity: 50.46%

Epoch 3/200 | Train Loss: 0.0049 | Train Acc: 99.94%
Validation Loss: 0.0478 | Validation Acc: 98.63% | Sparsity: 50.46%

New best model saved with Val Accuracy: 98.76%
Epoch 4/200 | Train Loss: 0.0019 | Train Acc: 99.98%
Validation Loss: 0.0473 | Validation Acc: 98.76% | Sparsity: 50.46%

New best model saved with Val Accuracy: 98.81%
Epoch 5/200 | Train Loss: 0.0012 | Train Acc: 99.99%
Validation Loss: 0.0452 | Validation Acc: 98.81% | Sparsity: 50.46%

New best model saved with Val Accuracy: 98.82%
Epoch 6/200 | Train Loss: 0.0008 | Train Acc: 100.00%
Validation Loss: 0.0462 | Validation Acc: 98.82% | Sparsity: 50.46%

Epoch 7/200 | Train Loss: 0.0006

In [117]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 95.49%


In [118]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [119]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=7.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 33s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [120]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=7.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5677 | Train Acc: 97.63% | Val Loss: 0.0573 | Val Acc: 98.25% | Sparsity: 50.46%
Epoch 2/50 | Train Loss: 0.3234 | Train Acc: 98.89% | Val Loss: 0.0483 | Val Acc: 98.47% | Sparsity: 50.46%
Epoch 3/50 | Train Loss: 0.2600 | Train Acc: 99.20% | Val Loss: 0.0537 | Val Acc: 98.26% | Sparsity: 50.46%
Epoch 4/50 | Train Loss: 0.2225 | Train Acc: 99.33% | Val Loss: 0.0456 | Val Acc: 98.73% | Sparsity: 50.46%
Epoch 5/50 | Train Loss: 0.1924 | Train Acc: 99.41% | Val Loss: 0.0468 | Val Acc: 98.54% | Sparsity: 50.46%
Epoch 6/50 | Train Loss: 0.1782 | Train Acc: 99.42% | Val Loss: 0.0491 | Val Acc: 98.51% | Sparsity: 50.46%
Epoch 7/50 | Train Loss: 0.1706 | Train Acc: 99.41% | Val Loss: 0.0450 | Val Acc: 98.58% | Sparsity: 50.46%
Epoch 8/50 | Train Loss: 0.1556 | Train Acc: 99.42% | Val Loss: 0.0443 | Val Acc: 98.69% | Sparsity: 50.46%
Epoch 9/50 | Train Loss: 0.1545 | Train Acc: 99.45% | Val Loss: 0.0470 | Val Acc: 98.56% | Sparsity: 50.46%
Early stopping triggered at 

In [121]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")

 Retrained Pruned Student Model Test Accuracy: 95.41%


In [130]:

model_path = 'student_before_pruning.pth'
# Load the model weights
student.load_state_dict(torch.load(model_path, map_location=torch.device('cpu')))

<All keys matched successfully>

In [131]:
# Pruning
print("Calculating Important Scores")
start_time = time.time()
importance_scores = compute_gradient_importance(
    teacher, student, train_loader, device, temperature=5.0, alpha=0.7, accumulation_epochs=3
)
total_time = time.time() - start_time
print(f"Total Time take to calculate Important scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")

print("Pruning the model")
start_time = time.time()

pruned_student = gradient_based_global_prune(student, importance_scores, prune_ratio=0.505)

total_time = time.time() - start_time
print(f"Total Time take to prune the model scores: {total_time // 60:.0f}m {total_time % 60:.0f}s")



Calculating Important Scores
Accumulation Epoch 1/3
Accumulation Epoch 2/3
Accumulation Epoch 3/3
Total Time take to calculate Important scores: 2m 31s
Pruning the model
Total Time take to prune the model scores: 0m 0s


In [ ]:

start_time = time.time()
pruned_student = retrain_with_KD(
    teacher, pruned_student, train_loader, val_loader,
    epochs=50, temperature=5.0, alpha=0.7, beta_prob=0.5,patience=5,save_path="pruned_student_retrain_KD_90%.pth"
)
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Retraining completed in {elapsed_time / 60:.2f} minutes ({elapsed_time:.2f} seconds)")

Epoch 1/50 | Train Loss: 0.5663 | Train Acc: 97.52% | Val Loss: 0.0652 | Val Acc: 98.01% | Sparsity: 50.46%
Epoch 2/50 | Train Loss: 0.3217 | Train Acc: 98.78% | Val Loss: 0.0512 | Val Acc: 98.43% | Sparsity: 50.46%
Epoch 3/50 | Train Loss: 0.2568 | Train Acc: 99.17% | Val Loss: 0.0521 | Val Acc: 98.30% | Sparsity: 50.46%
Epoch 4/50 | Train Loss: 0.2144 | Train Acc: 99.30% | Val Loss: 0.0494 | Val Acc: 98.60% | Sparsity: 50.46%
Epoch 5/50 | Train Loss: 0.1934 | Train Acc: 99.31% | Val Loss: 0.0479 | Val Acc: 98.58% | Sparsity: 50.46%
Epoch 6/50 | Train Loss: 0.1742 | Train Acc: 99.40% | Val Loss: 0.0498 | Val Acc: 98.47% | Sparsity: 50.46%
Epoch 7/50 | Train Loss: 0.1613 | Train Acc: 99.37% | Val Loss: 0.0477 | Val Acc: 98.57% | Sparsity: 50.46%
Epoch 8/50 | Train Loss: 0.1531 | Train Acc: 99.41% | Val Loss: 0.0473 | Val Acc: 98.62% | Sparsity: 50.46%
Epoch 9/50 | Train Loss: 0.1425 | Train Acc: 99.38% | Val Loss: 0.0519 | Val Acc: 98.38% | Sparsity: 50.46%
Epoch 10/50 | Train Loss: 0.

In [ ]:
student_accuracy = evaluate(pruned_student, test_loader, device)
print(f" Retrained Pruned Student Model Test Accuracy: {student_accuracy:.2f}%")